## RFC Generator using RNNs

Reference: https://github.com/martin-gorner/tensorflow-rnn-shakespeare

In [1]:
import tensorflow as tf
import numpy as np
import os
import glob
import my_txtutils as txt

%load_ext autoreload
%autoreload 2

In [2]:
print(tf.__version__)

1.3.0


In [3]:
# Read the RFC docs and generate a list of all characters, encode them

def process_rfc(dirpath, verbose=False):
    characters = list()
    
    for filename in glob.glob(os.path.join(dirpath, '*.txt')):
        if verbose:
            print('Reading filename', filename)
        l = open(filename, 'r', encoding='utf8').read()
        characters.extend(l)
    if verbose:
        print('Total length', len(characters))
        
    return characters

def encode(chars):
    characters = list()
    for c in chars:
        characters.append(ord(c))
    return characters

In [4]:
chars = encode(process_rfc('datasets/rfc'))
charset = list(set(chars))

In [5]:
# Generate training and validation data
train_pct = 95.0
sep = int(np.ceil((len(chars) * train_pct / float(100))))
chars_train = chars[:sep]
chars_valid = chars[sep:]

print('Training set length', len(chars_train))
print('Validation set length', len(chars_valid))

Training set length 25032430
Validation set length 1317496


In [6]:
################### CUSTOM
chars, valitext, bookranges = txt.read_data_files('datasets/rfc/*.txt', validation=False)
chars_train = chars
charset = list(set(chars))
################### CUSTOM

Loading file datasets/rfc/rfc1001.txt
Loading file datasets/rfc/rfc1002.txt
Loading file datasets/rfc/rfc1003.txt
Loading file datasets/rfc/rfc1004.txt
Loading file datasets/rfc/rfc1005.txt
Loading file datasets/rfc/rfc1006.txt
Loading file datasets/rfc/rfc1007.txt
Loading file datasets/rfc/rfc1008.txt
Loading file datasets/rfc/rfc1009.txt
Loading file datasets/rfc/rfc1010.txt
Loading file datasets/rfc/rfc1011.txt
Loading file datasets/rfc/rfc1012.txt
Loading file datasets/rfc/rfc1013.txt
Loading file datasets/rfc/rfc1014.txt
Loading file datasets/rfc/rfc1015.txt
Loading file datasets/rfc/rfc1016.txt
Loading file datasets/rfc/rfc1017.txt
Loading file datasets/rfc/rfc1018.txt
Loading file datasets/rfc/rfc1019.txt
Loading file datasets/rfc/rfc1020.txt
Loading file datasets/rfc/rfc1021.txt
Loading file datasets/rfc/rfc1022.txt
Loading file datasets/rfc/rfc1023.txt
Loading file datasets/rfc/rfc1024.txt
Loading file datasets/rfc/rfc1025.txt
Loading file datasets/rfc/rfc1026.txt
Loading file

Loading file datasets/rfc/rfc1229.txt
Loading file datasets/rfc/rfc1230.txt
Loading file datasets/rfc/rfc1231.txt
Loading file datasets/rfc/rfc1232.txt
Loading file datasets/rfc/rfc1233.txt
Loading file datasets/rfc/rfc1234.txt
Loading file datasets/rfc/rfc1235.txt
Loading file datasets/rfc/rfc1236.txt
Loading file datasets/rfc/rfc1237.txt
Loading file datasets/rfc/rfc1238.txt
Loading file datasets/rfc/rfc1239.txt
Loading file datasets/rfc/rfc1240.txt
Loading file datasets/rfc/rfc1241.txt
Loading file datasets/rfc/rfc1242.txt
Loading file datasets/rfc/rfc1243.txt
Loading file datasets/rfc/rfc1244.txt
Loading file datasets/rfc/rfc1245.txt
Loading file datasets/rfc/rfc1246.txt
Loading file datasets/rfc/rfc1247.txt
Loading file datasets/rfc/rfc1248.txt
Loading file datasets/rfc/rfc1249.txt
Loading file datasets/rfc/rfc1250.txt
Loading file datasets/rfc/rfc1251.txt
Loading file datasets/rfc/rfc1252.txt
Loading file datasets/rfc/rfc1253.txt
Loading file datasets/rfc/rfc1254.txt
Loading file

Loading file datasets/rfc/rfc1447.txt
Loading file datasets/rfc/rfc1448.txt
Loading file datasets/rfc/rfc1449.txt
Loading file datasets/rfc/rfc1450.txt
Loading file datasets/rfc/rfc1451.txt
Loading file datasets/rfc/rfc1452.txt
Loading file datasets/rfc/rfc1453.txt
Loading file datasets/rfc/rfc1454.txt
Loading file datasets/rfc/rfc1455.txt
Loading file datasets/rfc/rfc1456.txt
Loading file datasets/rfc/rfc1457.txt
Loading file datasets/rfc/rfc1458.txt
Loading file datasets/rfc/rfc1459.txt
Loading file datasets/rfc/rfc1460.txt
Loading file datasets/rfc/rfc1461.txt
Loading file datasets/rfc/rfc1462.txt
Loading file datasets/rfc/rfc1463.txt
Loading file datasets/rfc/rfc1464.txt
Loading file datasets/rfc/rfc1465.txt
Loading file datasets/rfc/rfc1466.txt
Loading file datasets/rfc/rfc1467.txt
Loading file datasets/rfc/rfc1468.txt
Loading file datasets/rfc/rfc1469.txt
Loading file datasets/rfc/rfc1470.txt
Loading file datasets/rfc/rfc1471.txt
Loading file datasets/rfc/rfc1472.txt
Loading file

In [7]:
VOCABSIZE = len(charset) # Our vocabulary size
SEQLEN = 30 # We will use a sequence size of 30 steps
BATCHSIZE = 500
NUM_NEURONS = 512
NUM_LAYERS = 3
LEARNING_RATE = 0.001
DROPOUT_PKEEP = 0.75
NUM_EPOCH = len(chars_train) // (BATCHSIZE * SEQLEN)
print('Num epochs', NUM_EPOCH)

Num epochs 1756


In [8]:
################### CUSTOM
VOCABSIZE = txt.ALPHASIZE
################### CUSTOM

In [9]:
# Place holders
x = tf.placeholder(tf.uint8, [None, None]) # [BATCHSIZE, SEQLEN], SEQLEN will be deduced from the input
x_ohe = tf.one_hot(x, VOCABSIZE, 1.0, 0.0)
y_actual = tf.placeholder(tf.uint8, [None, None]) # We are expecting the same input seq shifted by 1
y_actual_ohe = tf.one_hot(y_actual, VOCABSIZE, 1.0, 0.0)

h_initial_state = tf.placeholder(tf.float32, [None, NUM_NEURONS*NUM_LAYERS])

lr = tf.placeholder(tf.float32)
pkeep = tf.placeholder(tf.float32)
batchsize = tf.placeholder(tf.int32)

In [10]:
# The model, with dropouts

cells = [tf.contrib.rnn.GRUCell(NUM_NEURONS) for _ in range(NUM_LAYERS)]
dropcells = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=pkeep) for cell in cells]
multicell = tf.contrib.rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = tf.contrib.rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)

Yrnn, H = tf.nn.dynamic_rnn(multicell, x_ohe, dtype=tf.float32, initial_state=h_initial_state)
H = tf.identity(H, name='H')
# Shape of Yr is [BATCHSIZE, SEQLEN, NUM_NEURONS]
# Shape of H is [BATCHSIZE, SEQLEN * NUM_NEURONS]

print(cells[0].state_size)
print(multicell.state_size)
print(Yrnn.shape)
print(H.shape)

Yflat = tf.reshape(Yrnn, [-1, NUM_NEURONS]) # [BATCHSIZE * SEQLEN, NUM_NEURONS]

# Bring the actual and prediction to same shape, to compare
# This is our prediction
y_logits = tf.contrib.layers.linear(Yflat, VOCABSIZE) # [BATCHSIZE * SEQLEN, VOCABSIZE]
# This is actual/expected
y_actual_ohe_flat = tf.reshape(y_actual_ohe, [-1, VOCABSIZE]) # [BATCHSIZE * SEQLEN, VOCABSIZE]

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_logits, labels=y_actual_ohe_flat)
xentropy = tf.reshape(xentropy, [batchsize, -1]) # [BATCHSIZE, SEQLEN]

y_output_softmax = tf.nn.softmax(y_logits) # [BATCHSIZE * SEQLEN, VOCABSIZE]
y_output = tf.argmax(y_output_softmax, 1) # [BATCHSIZE * SEQLEN]
y_output = tf.reshape(y_output, [batchsize, -1]) # [BATCHSIZE, SEQLEN]

trainner = tf.train.AdamOptimizer(lr).minimize(xentropy)

512
1536
(?, ?, 512)
(?, 1536)


In [11]:
# To monitor progress

seqloss = tf.reduce_mean(xentropy, 1)
batchloss = tf.reduce_mean(seqloss)

equal = tf.equal(y_actual, tf.cast(y_output, tf.uint8))
accuracy = tf.reduce_mean(tf.cast(equal, tf.float32))

In [12]:
# Checkpoint location
if os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
saver = tf.train.Saver(max_to_keep=1000)

In [13]:

for x_, y_, epoch in txt.rnn_minibatch_sequencer(chars_train, BATCHSIZE, SEQLEN, 1):
    for a in x_:
        print(len(a), end='')
        for b in a:
            print(chr(b), end='')
        print('?', end='')
        break
        
    print('---------------------')
    for a in y_:
        print(len(a), end='')
        for b in a:
            print(chr(b), end='')
        print('?', end='')
        break
        
    break


30a0GVYQTM9QTMKPI)TQWRa4GSWGUV?---------------------
300GVYQTM9QTMKPI)TQWRa4GSWGUV?

In [14]:
# Now the training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_idx = 0
hinit = np.zeros([BATCHSIZE, NUM_NEURONS*NUM_LAYERS])

for x_, y_, epoch in txt.rnn_minibatch_sequencer(chars_train, BATCHSIZE, SEQLEN, 10):
    
    feed_dict = {x: x_, 
                 y_actual: y_, 
                 lr: LEARNING_RATE, 
                 pkeep: DROPOUT_PKEEP, 
                 h_initial_state: hinit, 
                 batchsize:BATCHSIZE}
    _, _y, _ostate = sess.run([trainner, y_output, H], feed_dict=feed_dict)
    hinit = _ostate
    
    print('Epoch', epoch, 'Iteration', batch_idx)
    batch_idx += 1

Epoch 0 Iteration 0
Epoch 0 Iteration 1
Epoch 0 Iteration 2
Epoch 0 Iteration 3
Epoch 0 Iteration 4
Epoch 0 Iteration 5
Epoch 0 Iteration 6
Epoch 0 Iteration 7
Epoch 0 Iteration 8
Epoch 0 Iteration 9
Epoch 0 Iteration 10
Epoch 0 Iteration 11
Epoch 0 Iteration 12
Epoch 0 Iteration 13
Epoch 0 Iteration 14
Epoch 0 Iteration 15
Epoch 0 Iteration 16
Epoch 0 Iteration 17
Epoch 0 Iteration 18
Epoch 0 Iteration 19
Epoch 0 Iteration 20
Epoch 0 Iteration 21
Epoch 0 Iteration 22
Epoch 0 Iteration 23
Epoch 0 Iteration 24
Epoch 0 Iteration 25
Epoch 0 Iteration 26
Epoch 0 Iteration 27
Epoch 0 Iteration 28
Epoch 0 Iteration 29
Epoch 0 Iteration 30
Epoch 0 Iteration 31
Epoch 0 Iteration 32
Epoch 0 Iteration 33
Epoch 0 Iteration 34
Epoch 0 Iteration 35
Epoch 0 Iteration 36
Epoch 0 Iteration 37
Epoch 0 Iteration 38
Epoch 0 Iteration 39
Epoch 0 Iteration 40
Epoch 0 Iteration 41
Epoch 0 Iteration 42
Epoch 0 Iteration 43
Epoch 0 Iteration 44
Epoch 0 Iteration 45
Epoch 0 Iteration 46
Epoch 0 Iteration 47
Ep

Epoch 0 Iteration 378
Epoch 0 Iteration 379
Epoch 0 Iteration 380
Epoch 0 Iteration 381
Epoch 0 Iteration 382
Epoch 0 Iteration 383
Epoch 0 Iteration 384
Epoch 0 Iteration 385
Epoch 0 Iteration 386
Epoch 0 Iteration 387
Epoch 0 Iteration 388
Epoch 0 Iteration 389
Epoch 0 Iteration 390
Epoch 0 Iteration 391
Epoch 0 Iteration 392
Epoch 0 Iteration 393
Epoch 0 Iteration 394
Epoch 0 Iteration 395
Epoch 0 Iteration 396
Epoch 0 Iteration 397
Epoch 0 Iteration 398
Epoch 0 Iteration 399
Epoch 0 Iteration 400
Epoch 0 Iteration 401
Epoch 0 Iteration 402
Epoch 0 Iteration 403
Epoch 0 Iteration 404
Epoch 0 Iteration 405
Epoch 0 Iteration 406
Epoch 0 Iteration 407
Epoch 0 Iteration 408
Epoch 0 Iteration 409
Epoch 0 Iteration 410
Epoch 0 Iteration 411
Epoch 0 Iteration 412
Epoch 0 Iteration 413
Epoch 0 Iteration 414
Epoch 0 Iteration 415
Epoch 0 Iteration 416
Epoch 0 Iteration 417
Epoch 0 Iteration 418
Epoch 0 Iteration 419
Epoch 0 Iteration 420
Epoch 0 Iteration 421
Epoch 0 Iteration 422
Epoch 0 It

Epoch 0 Iteration 752
Epoch 0 Iteration 753
Epoch 0 Iteration 754
Epoch 0 Iteration 755
Epoch 0 Iteration 756
Epoch 0 Iteration 757
Epoch 0 Iteration 758
Epoch 0 Iteration 759
Epoch 0 Iteration 760
Epoch 0 Iteration 761
Epoch 0 Iteration 762
Epoch 0 Iteration 763
Epoch 0 Iteration 764
Epoch 0 Iteration 765
Epoch 0 Iteration 766
Epoch 0 Iteration 767
Epoch 0 Iteration 768
Epoch 0 Iteration 769
Epoch 0 Iteration 770
Epoch 0 Iteration 771
Epoch 0 Iteration 772
Epoch 0 Iteration 773
Epoch 0 Iteration 774
Epoch 0 Iteration 775
Epoch 0 Iteration 776
Epoch 0 Iteration 777
Epoch 0 Iteration 778
Epoch 0 Iteration 779
Epoch 0 Iteration 780
Epoch 0 Iteration 781
Epoch 0 Iteration 782
Epoch 0 Iteration 783
Epoch 0 Iteration 784
Epoch 0 Iteration 785
Epoch 0 Iteration 786
Epoch 0 Iteration 787
Epoch 0 Iteration 788
Epoch 0 Iteration 789
Epoch 0 Iteration 790
Epoch 0 Iteration 791
Epoch 0 Iteration 792
Epoch 0 Iteration 793
Epoch 0 Iteration 794
Epoch 0 Iteration 795
Epoch 0 Iteration 796
Epoch 0 It

Epoch 0 Iteration 1120
Epoch 0 Iteration 1121
Epoch 0 Iteration 1122
Epoch 0 Iteration 1123
Epoch 0 Iteration 1124
Epoch 0 Iteration 1125
Epoch 0 Iteration 1126
Epoch 0 Iteration 1127
Epoch 0 Iteration 1128
Epoch 0 Iteration 1129
Epoch 0 Iteration 1130
Epoch 0 Iteration 1131
Epoch 0 Iteration 1132
Epoch 0 Iteration 1133
Epoch 0 Iteration 1134
Epoch 0 Iteration 1135
Epoch 0 Iteration 1136
Epoch 0 Iteration 1137
Epoch 0 Iteration 1138
Epoch 0 Iteration 1139
Epoch 0 Iteration 1140
Epoch 0 Iteration 1141
Epoch 0 Iteration 1142
Epoch 0 Iteration 1143
Epoch 0 Iteration 1144
Epoch 0 Iteration 1145
Epoch 0 Iteration 1146
Epoch 0 Iteration 1147
Epoch 0 Iteration 1148
Epoch 0 Iteration 1149
Epoch 0 Iteration 1150
Epoch 0 Iteration 1151
Epoch 0 Iteration 1152
Epoch 0 Iteration 1153
Epoch 0 Iteration 1154
Epoch 0 Iteration 1155
Epoch 0 Iteration 1156
Epoch 0 Iteration 1157
Epoch 0 Iteration 1158
Epoch 0 Iteration 1159
Epoch 0 Iteration 1160
Epoch 0 Iteration 1161
Epoch 0 Iteration 1162
Epoch 0 Ite

Epoch 0 Iteration 1478
Epoch 0 Iteration 1479
Epoch 0 Iteration 1480
Epoch 0 Iteration 1481
Epoch 0 Iteration 1482
Epoch 0 Iteration 1483
Epoch 0 Iteration 1484
Epoch 0 Iteration 1485
Epoch 0 Iteration 1486
Epoch 0 Iteration 1487
Epoch 0 Iteration 1488
Epoch 0 Iteration 1489
Epoch 0 Iteration 1490
Epoch 0 Iteration 1491
Epoch 0 Iteration 1492
Epoch 0 Iteration 1493
Epoch 0 Iteration 1494
Epoch 0 Iteration 1495
Epoch 0 Iteration 1496
Epoch 0 Iteration 1497
Epoch 0 Iteration 1498
Epoch 0 Iteration 1499
Epoch 0 Iteration 1500
Epoch 0 Iteration 1501
Epoch 0 Iteration 1502
Epoch 0 Iteration 1503
Epoch 0 Iteration 1504
Epoch 0 Iteration 1505
Epoch 0 Iteration 1506
Epoch 0 Iteration 1507
Epoch 0 Iteration 1508
Epoch 0 Iteration 1509
Epoch 0 Iteration 1510
Epoch 0 Iteration 1511
Epoch 0 Iteration 1512
Epoch 0 Iteration 1513
Epoch 0 Iteration 1514
Epoch 0 Iteration 1515
Epoch 0 Iteration 1516
Epoch 0 Iteration 1517
Epoch 0 Iteration 1518
Epoch 0 Iteration 1519
Epoch 0 Iteration 1520
Epoch 0 Ite

...

Epoch 9 Iteration 15880
Epoch 9 Iteration 15881
Epoch 9 Iteration 15882
Epoch 9 Iteration 15883
Epoch 9 Iteration 15884
Epoch 9 Iteration 15885
Epoch 9 Iteration 15886
Epoch 9 Iteration 15887
Epoch 9 Iteration 15888
Epoch 9 Iteration 15889
Epoch 9 Iteration 15890
Epoch 9 Iteration 15891
Epoch 9 Iteration 15892
Epoch 9 Iteration 15893
Epoch 9 Iteration 15894
Epoch 9 Iteration 15895
Epoch 9 Iteration 15896
Epoch 9 Iteration 15897
Epoch 9 Iteration 15898
Epoch 9 Iteration 15899
Epoch 9 Iteration 15900
Epoch 9 Iteration 15901
Epoch 9 Iteration 15902
Epoch 9 Iteration 15903
Epoch 9 Iteration 15904
Epoch 9 Iteration 15905
Epoch 9 Iteration 15906
Epoch 9 Iteration 15907
Epoch 9 Iteration 15908
Epoch 9 Iteration 15909
Epoch 9 Iteration 15910
Epoch 9 Iteration 15911
Epoch 9 Iteration 15912
Epoch 9 Iteration 15913
Epoch 9 Iteration 15914
Epoch 9 Iteration 15915
Epoch 9 Iteration 15916
Epoch 9 Iteration 15917
Epoch 9 Iteration 15918
Epoch 9 Iteration 15919
Epoch 9 Iteration 15920
Epoch 9 Iteratio

Epoch 9 Iteration 16222
Epoch 9 Iteration 16223
Epoch 9 Iteration 16224
Epoch 9 Iteration 16225
Epoch 9 Iteration 16226
Epoch 9 Iteration 16227
Epoch 9 Iteration 16228
Epoch 9 Iteration 16229
Epoch 9 Iteration 16230
Epoch 9 Iteration 16231
Epoch 9 Iteration 16232
Epoch 9 Iteration 16233
Epoch 9 Iteration 16234
Epoch 9 Iteration 16235
Epoch 9 Iteration 16236
Epoch 9 Iteration 16237
Epoch 9 Iteration 16238
Epoch 9 Iteration 16239
Epoch 9 Iteration 16240
Epoch 9 Iteration 16241
Epoch 9 Iteration 16242
Epoch 9 Iteration 16243
Epoch 9 Iteration 16244
Epoch 9 Iteration 16245
Epoch 9 Iteration 16246
Epoch 9 Iteration 16247
Epoch 9 Iteration 16248
Epoch 9 Iteration 16249
Epoch 9 Iteration 16250
Epoch 9 Iteration 16251
Epoch 9 Iteration 16252
Epoch 9 Iteration 16253
Epoch 9 Iteration 16254
Epoch 9 Iteration 16255
Epoch 9 Iteration 16256
Epoch 9 Iteration 16257
Epoch 9 Iteration 16258
Epoch 9 Iteration 16259
Epoch 9 Iteration 16260
Epoch 9 Iteration 16261
Epoch 9 Iteration 16262
Epoch 9 Iteratio

Epoch 9 Iteration 16564
Epoch 9 Iteration 16565
Epoch 9 Iteration 16566
Epoch 9 Iteration 16567
Epoch 9 Iteration 16568
Epoch 9 Iteration 16569
Epoch 9 Iteration 16570
Epoch 9 Iteration 16571
Epoch 9 Iteration 16572
Epoch 9 Iteration 16573
Epoch 9 Iteration 16574
Epoch 9 Iteration 16575
Epoch 9 Iteration 16576
Epoch 9 Iteration 16577
Epoch 9 Iteration 16578
Epoch 9 Iteration 16579
Epoch 9 Iteration 16580
Epoch 9 Iteration 16581
Epoch 9 Iteration 16582
Epoch 9 Iteration 16583
Epoch 9 Iteration 16584
Epoch 9 Iteration 16585
Epoch 9 Iteration 16586
Epoch 9 Iteration 16587
Epoch 9 Iteration 16588
Epoch 9 Iteration 16589
Epoch 9 Iteration 16590
Epoch 9 Iteration 16591
Epoch 9 Iteration 16592
Epoch 9 Iteration 16593
Epoch 9 Iteration 16594
Epoch 9 Iteration 16595
Epoch 9 Iteration 16596
Epoch 9 Iteration 16597
Epoch 9 Iteration 16598
Epoch 9 Iteration 16599
Epoch 9 Iteration 16600
Epoch 9 Iteration 16601
Epoch 9 Iteration 16602
Epoch 9 Iteration 16603
Epoch 9 Iteration 16604
Epoch 9 Iteratio

Epoch 9 Iteration 16906
Epoch 9 Iteration 16907
Epoch 9 Iteration 16908
Epoch 9 Iteration 16909
Epoch 9 Iteration 16910
Epoch 9 Iteration 16911
Epoch 9 Iteration 16912
Epoch 9 Iteration 16913
Epoch 9 Iteration 16914
Epoch 9 Iteration 16915
Epoch 9 Iteration 16916
Epoch 9 Iteration 16917
Epoch 9 Iteration 16918
Epoch 9 Iteration 16919
Epoch 9 Iteration 16920
Epoch 9 Iteration 16921
Epoch 9 Iteration 16922
Epoch 9 Iteration 16923
Epoch 9 Iteration 16924
Epoch 9 Iteration 16925
Epoch 9 Iteration 16926
Epoch 9 Iteration 16927
Epoch 9 Iteration 16928
Epoch 9 Iteration 16929
Epoch 9 Iteration 16930
Epoch 9 Iteration 16931
Epoch 9 Iteration 16932
Epoch 9 Iteration 16933
Epoch 9 Iteration 16934
Epoch 9 Iteration 16935
Epoch 9 Iteration 16936
Epoch 9 Iteration 16937
Epoch 9 Iteration 16938
Epoch 9 Iteration 16939
Epoch 9 Iteration 16940
Epoch 9 Iteration 16941
Epoch 9 Iteration 16942
Epoch 9 Iteration 16943
Epoch 9 Iteration 16944
Epoch 9 Iteration 16945
Epoch 9 Iteration 16946
Epoch 9 Iteratio

Epoch 9 Iteration 17248
Epoch 9 Iteration 17249
Epoch 9 Iteration 17250
Epoch 9 Iteration 17251
Epoch 9 Iteration 17252
Epoch 9 Iteration 17253
Epoch 9 Iteration 17254
Epoch 9 Iteration 17255
Epoch 9 Iteration 17256
Epoch 9 Iteration 17257
Epoch 9 Iteration 17258
Epoch 9 Iteration 17259
Epoch 9 Iteration 17260
Epoch 9 Iteration 17261
Epoch 9 Iteration 17262
Epoch 9 Iteration 17263
Epoch 9 Iteration 17264
Epoch 9 Iteration 17265
Epoch 9 Iteration 17266
Epoch 9 Iteration 17267
Epoch 9 Iteration 17268
Epoch 9 Iteration 17269
Epoch 9 Iteration 17270
Epoch 9 Iteration 17271
Epoch 9 Iteration 17272
Epoch 9 Iteration 17273
Epoch 9 Iteration 17274
Epoch 9 Iteration 17275
Epoch 9 Iteration 17276
Epoch 9 Iteration 17277
Epoch 9 Iteration 17278
Epoch 9 Iteration 17279
Epoch 9 Iteration 17280
Epoch 9 Iteration 17281
Epoch 9 Iteration 17282
Epoch 9 Iteration 17283
Epoch 9 Iteration 17284
Epoch 9 Iteration 17285
Epoch 9 Iteration 17286
Epoch 9 Iteration 17287
Epoch 9 Iteration 17288
Epoch 9 Iteratio

In [25]:
txt.print_text_generation_header()
ry = np.array([[txt.convert_from_alphabet(ord("b"))]])
rh = np.zeros([1, NUM_NEURONS * NUM_LAYERS])
for k in range(2000):
    ryo, rh = sess.run([y_output_softmax, H], feed_dict={x: ry, pkeep: 1.0, h_initial_state: rh, batchsize: 1})
    #print(ryo.shape)
    rc = txt.sample_from_probabilities(ryo, 10)
    print(chr(txt.convert_to_alphabet(rc)), end="")
    ry = np.array([[rc]])
txt.print_text_generation_footer()


┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
 south err
   -= Bu stutt marcedruns dutaet lutsent tam sean  n uno uu bu re at th to
  so ba us le aa ou re ro re ir su sets  ro ut
  r  su up tl ro re,dh tr uh th us vu sour us
  &comb2 "  SP a  E E  c C' I A? e o! I o? O U' O E- u O' U U'
  &r   12 Se UM ST AC S' A  U! US I- O: ': U! O'
  -! O- NA O' O: U' U? u! U/ U' NT OU U:
  U! U: D- U/ Ca U' DH D. Te To Ce Wu Ni Re S1 C6 NA AA A' S'
  S? CA A' O? N/ U' O/ E:
  D/ a: i! i! i' i> I: i! U' I! i' O?
  S! U, A' E? O: I! I! Y' I: U: O! E< I: E!
  &! "! a' O? O/
  Ca AS DO N  A  N  G  K  H  U  Z  ?
  ?: ?! "' '. '! ?? ?? ?? ?? '- ?? ??
  CS O? A? N? BH OH EG ES OS GM US PC SC UC Co Ca '1 "6 O' O: U! o! O' U:
  &code6 a "S -- CS AT E' E' E' E! I! I! I: A! A!
  S. "/ "! ": o: E! o. U! U! O?
  &/ i! "? A: '! ": O' u: e' U! I! A! O'
  !! '' '> ', '? ', '' ', ",
  "> "? A! ", '' -, O> O> O: U? I- O:
  & omb2 "! NO -4 -  C  A, 